# Layers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mitchell-Mirano/sorix/blob/qa/docs/learn/04-layers.ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-black?logo=github)](https://github.com/Mitchell-Mirano/sorix/blob/qa/docs/learn/04-layers.ipynb)
[![Open in Docs](https://img.shields.io/badge/Open%20in-Docs-blue?logo=readthedocs)](http://127.0.0.1:8000/learn/04-layers/)



The layers are implemented as Python classes that encapsulate one or more fundamental tensor operations. Each layer defines its trainable parameters as class attributes represented by `tensor` objects, which are initialized with `requires_grad=True` by default to enable automatic differentiation.

These layers are designed to be composable and device-aware, supporting execution on both CPU and GPU backends. Parameter initialization follows standard schemes (e.g., He or Xavier initialization), and all learnable parameters are exposed through a unified interface for optimization. Non-trainable quantities, such as running statistics in normalization layers, are handled as buffers and are therefore excluded from gradient computation.

At a high level, the available layers include linear transformations, nonlinear activation functions, and normalization modules. Each class defines the forward computation via the `__call__` interface, while gradient propagation is handled internally through the underlying tensor autograd mechanism. Detailed mathematical formulations and implementation specifics of each layer are discussed in the following sections.


In [2]:
# Uncomment the next line and run this cell to install sorix
#!pip install 'sorix @ git+https://github.com/Mitchell-Mirano/sorix.git@qa'

## Import Sorix

In [3]:
import numpy as np
from sorix import tensor
from sorix.nn import ReLU,Linear,BatchNorm1d
import sorix

## Linear

The **Linear** layer implements an affine transformation between finite-dimensional real vector spaces and constitutes a fundamental operator in deep learning architectures. Formally, it defines a linear mapping from an input feature space to an output representation space, optionally augmented by a bias term. This transformation is applied independently to each element of a batch.

### Mathematical definition

Let  $\mathbf{X} \in \mathbb{R}^{N \times d}$
be an input tensor representing a batch of $N$ samples, where each sample is a vector in a $d$-dimensional feature space. The Linear layer defines the affine transformation

$$\mathbf{Y} = \mathbf{X}\mathbf{W} + \mathbf{b}$$

where the involved quantities have the following dimensions:

- $
  \mathbf{X} \in \mathbb{R}^{N \times d}
  $ : input batch matrix  
- $
  \mathbf{W} \in \mathbb{R}^{d \times m}
  $ : weight matrix (trainable parameters)  
- $
  \mathbf{b} \in \mathbb{R}^{1 \times m}
  $ : bias vector associated with the output neurons  
- $
  \mathbf{Y} \in \mathbb{R}^{N \times m}
  $ : output tensor  
- $
  m
  $ : number of neurons, i.e., the dimensionality of the output space  

From a dimensional analysis standpoint, the matrix product

$$
\mathbf{X}\mathbf{W} :
\mathbb{R}^{N \times d} \times \mathbb{R}^{d \times m}
\;\longrightarrow\;
\mathbb{R}^{N \times m}
$$

is well-defined. The bias term $\mathbf{b}$ is then added **column-wise** to the resulting matrix, meaning that each component $b_j$ is added to all entries of the $j$-th output column. Explicitly,

$$
Y_{ij} = (\mathbf{X}\mathbf{W})_{ij} + b_j,
\quad
i = 1,\dots,N,\;
j = 1,\dots,m.
$$

### Interpretation as a linear mapping

At the level of individual samples, for each
$
i \in \{1, \dots, N\},
$
the transformation can be written as

$$
\mathbf{y}_i = \mathbf{x}_i \mathbf{W} + \mathbf{b},
\quad
\mathbf{x}_i \in \mathbb{R}^{1 \times d},\;
\mathbf{y}_i \in \mathbb{R}^{1 \times m}.
$$

Thus, each output vector $\mathbf{y}_i$ is obtained as a linear combination of the input features, defined by the columns of $\mathbf{W}$, followed by a translation in the output space determined by the bias vector $\mathbf{b}$.

### Functional view

The Linear layer realizes the mapping

$$
\text{Linear}:\;
\mathbb{R}^{N \times d}
\;\longrightarrow\;
\mathbb{R}^{N \times m},
$$

where the same affine transformation is applied independently to each sample in the batch. This operator forms the mathematical foundation upon which more complex nonlinear models are constructed when composed with activation and normalization layers.

### Parameterization and gradients

The parameters $\mathbf{W}$ and $\mathbf{b}$ are represented as `tensor` objects with `requires_grad=True`, enabling automatic gradient computation via automatic differentiation. During backpropagation, the following gradients are computed:

$$
\frac{\partial \mathcal{L}}{\partial \mathbf{W}}, \quad
\frac{\partial \mathcal{L}}{\partial \mathbf{b}}, \quad
\frac{\partial \mathcal{L}}{\partial \mathbf{X}},
$$

where $\mathcal{L}$ denotes the global loss function of the model.

### Parameter initialization

The weight matrix $\mathbf{W}$ is initialized from a zero-mean normal distribution with a standard deviation determined by the chosen initialization scheme:

- **He initialization** (recommended for ReLU-like activations):
  $$
  \sigma = \sqrt{\frac{2}{d}}.
  $$

- **Xavier initialization** (suitable for symmetric activations such as $\tanh$):
  $$
  \sigma = \sqrt{\frac{2}{d + m}}.
  $$

Formally,
$$
W_{ij} \sim \mathcal{N}(0, \sigma^2).
$$

When present, the bias vector $\mathbf{b}$ is initialized to zero.

### Forward computation

Given an input tensor $\mathbf{X}$, the forward evaluation of the layer is performed through the matrix operation

$$
\text{Linear}(\mathbf{X}) = \mathbf{X}\mathbf{W} + \mathbf{b}.
$$

In the implementation, this computation is exposed via the `__call__` method, enabling a concise and functional syntax consistent with the rest of the framework.

### Multi-device support

The Linear layer is device-aware. Parameters and computations may reside on either CPU or GPU, using NumPy or CuPy as the numerical backend, respectively. The `to(device)` method ensures consistent parameter transfer across devices while preserving the mathematical semantics of the transformation.

### Parameter interface

The trainable parameters of the layer are exposed through the `parameters()` method, which returns the set

$$
\{\mathbf{W}, \mathbf{b}\},
$$

or only $\mathbf{W}$ when the bias term is disabled. This abstraction allows direct integration with gradient-based optimization algorithms.

### Statistical interpretation

From a statistical perspective, the Linear layer can be interpreted as a multivariate linear regression model, where each output neuron represents a linear combination of the input features. In this context, the coefficients of the weight matrix $\mathbf{W}$ and the bias vector $\mathbf{b}$ define hyperplanes in the output space that approximate the relationship between input and output variables.


In [32]:
# create random input data
samples = 10
features = 3
neurons = 2

# X ∈ ℝ^(samples × features)
X = tensor(np.random.randn(samples, features))
X

Tensor(
[[-2.11857762  1.68792676  1.31994672]
 [ 1.5472385   0.1157047  -0.47949683]
 [-0.60946832 -0.26404178  1.0974149 ]
 [-0.02944926  1.27697084  0.55260467]
 [-1.14989104  1.20767261  0.87711009]
 [-0.20930424 -1.50447589  1.70364656]
 [-0.52887809 -2.09572447  1.52516325]
 [ 0.33512794  1.30232238  2.43443063]
 [ 1.12924792 -0.34338007  0.09819372]
 [-3.17633068 -0.30102419  1.39405847]], shape=(10, 3), device=cpu, requires_grad=False)

In [33]:
# instantiate a Linear layer: ℝ^(samples × features) → ℝ^(samples × neurons)
linear = Linear(features, neurons)

# weight matrix W ∈ ℝ^(features × neurons)
print(linear.W)

# bias vector b ∈ ℝ^(1 × neurons)
print(linear.b)

Tensor(
[[-0.07543034  0.36379951]
 [-0.64783009 -1.21172978]
 [ 0.53614753  1.63158125]], shape=(3, 2), device=cpu, requires_grad=True)
Tensor(
[[0. 0.]], shape=(1, 2), device=cpu, requires_grad=True)


In [34]:
# forward pass:
# Y ∈ ℝ^(samples × neurons) = X @ W + b
Y = linear(X)
print(Y)


Tensor(
[[-0.22599853 -0.66244831]
 [-0.44874675 -0.35965625]
 [ 0.8054029   1.88874458]
 [-0.52876113 -0.6564378 ]
 [-0.22536957 -0.45062629]
 [ 1.90383853  4.52651126]
 [ 2.21527932  4.83547393]
 [ 0.43625153  2.51582794]
 [ 0.18991871  0.98711474]
 [ 1.18202524  1.48373209]], shape=(10, 2), device=cpu, requires_grad=True)


## Batch Normalization (BatchNorm1d)

The **BatchNorm1d** layer implements *batch normalization*, a technique designed to stabilize and accelerate the training of deep neural networks by reducing internal covariate shift. This is achieved by normalizing intermediate activations across the batch dimension and subsequently applying a learnable affine transformation.

### Mathematical definition

Let $ \mathbf{X} \in \mathbb{R}^{N \times d} $
be an input tensor representing a batch of $N$ samples, where each sample has $d$ features. Batch normalization operates **feature-wise**, normalizing each feature independently across the batch.

During training, the batch-wise mean and variance are computed as

$$
\boldsymbol{\mu}_B
=
\frac{1}{N}
\sum_{i=1}^{N}
\mathbf{x}_i
\;\in\;
\mathbb{R}^{1 \times d},
$$

$$
\boldsymbol{\sigma}_B^2
=
\frac{1}{N}
\sum_{i=1}^{N}
(\mathbf{x}_i - \boldsymbol{\mu}_B)^2
\;\in\;
\mathbb{R}^{1 \times d},
$$

where $\mathbf{x}_i \in \mathbb{R}^{1 \times d}$ denotes the $i$-th sample in the batch.

### Normalization step

Each input sample is normalized using the batch statistics:

$$
\widehat{\mathbf{X}}
=
\frac{\mathbf{X} - \boldsymbol{\mu}_B}
{\sqrt{\boldsymbol{\sigma}_B^2 + \varepsilon}},
\quad
\widehat{\mathbf{X}} \in \mathbb{R}^{N \times d},
$$

where $\varepsilon > 0$ is a small constant introduced for numerical stability.

### Learnable affine transformation

To preserve the representational capacity of the network, batch normalization introduces two learnable parameters:

- Scale parameter:
  $
  \boldsymbol{\gamma} \in \mathbb{R}^{1 \times d}
  $
- Shift parameter:
  $
  \boldsymbol{\beta} \in \mathbb{R}^{1 \times d}
  $

The final output of the layer is given by

$$
\mathbf{Y}
=
\boldsymbol{\gamma} \odot \widehat{\mathbf{X}}
+
\boldsymbol{\beta},
\quad
\mathbf{Y} \in \mathbb{R}^{N \times d},
$$

where $\odot$ denotes element-wise multiplication applied **column-wise**, i.e., independently to each feature.

### Running statistics and inference mode

In addition to batch statistics, BatchNorm1d maintains *running estimates* of the mean and variance:

$$
\boldsymbol{\mu}_{\text{run}} \in \mathbb{R}^{1 \times d},
\quad
\boldsymbol{\sigma}^2_{\text{run}} \in \mathbb{R}^{1 \times d}.
$$

These statistics are updated during training using an exponential moving average:

$$
\boldsymbol{\mu}_{\text{run}}
\leftarrow
\alpha \boldsymbol{\mu}_{\text{run}}
+
(1 - \alpha)\boldsymbol{\mu}_B,
$$

$$
\boldsymbol{\sigma}^2_{\text{run}}
\leftarrow
\alpha \boldsymbol{\sigma}^2_{\text{run}}
+
(1 - \alpha)\boldsymbol{\sigma}^2_B,
$$

where $\alpha \in (0,1)$ is the momentum parameter controlling the update rate.

During inference (evaluation mode), normalization is performed using these accumulated running statistics instead of the batch statistics, ensuring deterministic behavior:

$$
\widehat{\mathbf{X}}
=
\frac{\mathbf{X} - \boldsymbol{\mu}_{\text{run}}}
{\sqrt{\boldsymbol{\sigma}^2_{\text{run}} + \varepsilon}}.
$$

### Functional view

The BatchNorm1d layer realizes the mapping

$$
\text{BatchNorm1d}:\;
\mathbb{R}^{N \times d}
\;\longrightarrow\;
\mathbb{R}^{N \times d},
$$

where normalization and affine reparameterization are applied independently to each feature across the batch.

### Parameterization and gradients

The learnable parameters $\boldsymbol{\gamma}$ and $\boldsymbol{\beta}$ are represented as `tensor` objects with `requires_grad=True`. Gradients are computed with respect to these parameters, as well as with respect to the input tensor $\mathbf{X}$, during backpropagation:

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{\gamma}}, \quad
\frac{\partial \mathcal{L}}{\partial \boldsymbol{\beta}}, \quad
\frac{\partial \mathcal{L}}{\partial \mathbf{X}}.
$$

The running statistics are treated as buffers and do not participate in gradient computation.

### Multi-device support

BatchNorm1d is device-aware and supports execution on CPU and GPU backends. Learnable parameters are stored as tensors on the selected device, while running statistics are maintained as NumPy or CuPy arrays and transferred consistently when changing devices via the `to(device)` method.

### Parameter interface

The trainable parameters of the layer are exposed through the `parameters()` method, which returns

$$
\{\boldsymbol{\gamma}, \boldsymbol{\beta}\}.
$$

### Statistical interpretation

From a statistical perspective, BatchNorm1d performs a feature-wise standardization of the input distribution, followed by a learned affine transformation. This can be interpreted as dynamically re-centering and re-scaling the feature space, which improves numerical conditioning and facilitates optimization in deep networks.


In [38]:
# number of samples and features
samples = 8
features = 3

# input tensor X ∈ ℝ^(samples × features)
X = tensor(np.random.randn(samples, features))
X


Tensor(
[[ 1.22129142 -0.0213213  -0.29432569]
 [-0.91636576 -0.06141393 -1.13231409]
 [-2.61212584 -0.89636751 -0.9959895 ]
 [-1.6714919  -1.37353929  0.9229026 ]
 [ 0.89873948 -0.36702833 -0.64483109]
 [ 0.52349229 -0.67205028 -0.62714142]
 [-0.14046412  0.33438818 -0.96513357]
 [ 0.80115808  1.47316211  0.29733451]], shape=(8, 3), device=cpu, requires_grad=False)

In [39]:
bn = BatchNorm1d(features)

# γ ∈ ℝ^(1 × features), β ∈ ℝ^(1 × features)
print(bn.gamma)
print(bn.beta)


Tensor(
[[1. 1. 1.]], shape=(1, 3), device=cpu, requires_grad=True)
Tensor(
[[0. 0. 0.]], shape=(1, 3), device=cpu, requires_grad=True)


In [40]:
# forward pass (training mode)
Y = bn(X)
Y


Tensor(
[[ 1.1334296   0.21814242  0.20320982]
 [-0.52805754  0.16864665 -1.05249018]
 [-1.8460816  -0.86213323 -0.84821195]
 [-1.1149769  -1.45121875  2.02718921]
 [ 0.88272715 -0.20864519 -0.32201181]
 [ 0.59106748 -0.58520564 -0.29550437]
 [ 0.0750095   0.65727842 -0.80197528]
 [ 0.80688232  2.06313531  1.08979455]], shape=(8, 3), device=cpu, requires_grad=True)

In [41]:
# running statistics after the forward pass
print(bn.running_mean)
print(bn.running_var)


[[-0.02369708 -0.01980213 -0.04299373]]
[[1.06553105 0.96561242 0.94453426]]


In [42]:
# inference mode
bn.training = False
Y_eval = bn(X)
Y_eval


Tensor(
[[ 1.20609147e+00 -1.54597652e-03 -2.58604792e-01]
 [-8.64779137e-01 -4.23460407e-02 -1.12084217e+00]
 [-2.50755880e+00 -8.92032437e-01 -9.80572746e-01]
 [-1.59631297e+00 -1.37762394e+00  9.93846602e-01]
 [ 8.93616984e-01 -3.53353027e-01 -6.19252818e-01]
 [ 5.30093599e-01 -6.63757116e-01 -6.01051251e-01]
 [-1.13118899e-01  3.60440022e-01 -9.48823932e-01]
 [ 7.99084306e-01  1.51930769e+00  3.50176361e-01]], shape=(8, 3), device=cpu, requires_grad=True)

## ReLU (Rectified Linear Unit)

The **ReLU** layer implements the *Rectified Linear Unit* activation function, one of the most widely used nonlinearities in deep learning due to its simplicity, computational efficiency, and favorable gradient properties. ReLU introduces nonlinearity by applying an element-wise thresholding operation to its input.

### Mathematical definition

Let  
$
\mathbf{X} \in \mathbb{R}^{N \times d}
$
be an input tensor representing a batch of $N$ samples with $d$ features. The ReLU activation is defined element-wise as

$$
\operatorname{ReLU}(x) = \max(0, x).
$$

Applied to a tensor, this yields

$$
\mathbf{Y} = \operatorname{ReLU}(\mathbf{X}),
\quad
Y_{ij} = \max(0, X_{ij}),
$$

where

- $
  \mathbf{Y} \in \mathbb{R}^{N \times d}
  $
  is the output tensor,
- $
  i = 1,\dots,N
  $
  indexes the samples,
- $
  j = 1,\dots,d
  $
  indexes the features.

The transformation preserves the dimensionality of the input:

$$
\text{ReLU}:\;
\mathbb{R}^{N \times d}
\;\longrightarrow\;
\mathbb{R}^{N \times d}.
$$

### Forward computation

In the forward pass, ReLU performs a simple element-wise comparison with zero, retaining positive values and setting negative values to zero. This operation is computationally inexpensive and trivially parallelizable.

Formally,

$$
\mathbf{Y} = \max(\mathbf{0}, \mathbf{X}),
$$

where the maximum is taken element-wise.

### Backward computation (gradient)

The derivative of the ReLU function is given by

$$
\frac{d}{dx} \operatorname{ReLU}(x)
=
\begin{cases}
1, & x > 0, \\
0, & x \le 0.
\end{cases}
$$

Accordingly, during backpropagation, the gradient with respect to the input tensor is computed as

$$
\frac{\partial \mathcal{L}}{\partial X_{ij}}
=
\frac{\partial \mathcal{L}}{\partial Y_{ij}}
\cdot
\mathbb{I}(X_{ij} > 0),
$$

where $\mathbb{I}(\cdot)$ denotes the indicator function.

This matches the implementation, where the upstream gradient is masked by the condition $X > 0$.

### Autograd implementation details

In the provided implementation, the output tensor stores a reference to the input tensor and defines a custom backward function:

- The forward pass computes $\mathbf{Y} = \max(0, \mathbf{X})$.
- The backward pass accumulates gradients in the input tensor via
  $
  X.\text{grad} \mathrel{+}= \text{out.grad} \cdot (X > 0).
  $

This ensures correct gradient flow through the activation during optimization.

### Device awareness

The ReLU layer is device-aware and supports execution on both CPU and GPU. The numerical backend (NumPy or CuPy) is selected dynamically based on the device associated with the input tensor, ensuring consistency and performance portability.

### Functional role in neural networks

From a functional standpoint, ReLU introduces nonlinearity while preserving sparsity in activations, as negative inputs are mapped exactly to zero. This property mitigates the vanishing gradient problem commonly observed with saturating nonlinearities and enables efficient training of deep architectures.


In [44]:
# number of samples and features
samples = 8
features = 3

# input tensor X ∈ ℝ^(samples × features)
X = tensor(np.random.randn(samples, features))
X

Tensor(
[[-0.89057293 -0.26609062 -0.07709271]
 [-0.32024358 -1.6020258   0.18186056]
 [ 0.06751868  0.94575422 -0.27170854]
 [-0.93181959  0.22443158 -1.08736002]
 [-1.66797155 -0.89579603  2.077023  ]
 [ 0.68590973 -2.62432205  1.08619278]
 [ 0.38510981 -0.49339594 -0.00567828]
 [-0.34719925  0.55854604 -0.53541034]], shape=(8, 3), device=cpu, requires_grad=False)

In [45]:
relu = ReLU()
ReLU(X)

Tensor(
[[0.         0.         0.        ]
 [0.         0.         0.18186056]
 [0.06751868 0.94575422 0.        ]
 [0.         0.22443158 0.        ]
 [0.         0.         2.077023  ]
 [0.68590973 0.         1.08619278]
 [0.38510981 0.         0.        ]
 [0.         0.55854604 0.        ]], shape=(8, 3), device=cpu, requires_grad=False)

## ReLu + BatchNorm1d + Linear

In [56]:
# number of samples and features
samples = 8
features = 3
neurons = 2

# input tensor X ∈ ℝ^(samples × features)
X = tensor(np.random.randn(samples, features))

#Linear: X ∈ ℝ^(samples × features) -> X ∈ ℝ^(samples × neurons)
linear = Linear(features,neurons)

#BatchNorm1d: X ∈ ℝ^(samples × neurons) -> X ∈ ℝ^(samples × neurons)
bn = BatchNorm1d(neurons)

#Relu: X ∈ ℝ^(samples × neurons) -> X ∈ ℝ^(samples × neurons)
relu = ReLU()

Y = ReLU(bn(linear(X)))
Y

Tensor(
[[0.7899231  0.        ]
 [0.76963545 0.        ]
 [0.         1.27633776]
 [0.59711683 0.        ]
 [0.69790922 0.6917095 ]
 [0.62584084 0.        ]
 [0.         0.88076543]
 [0.         0.81695402]], shape=(8, 2), device=cpu, requires_grad=True)

## Using GPU

In [57]:
device = 'gpu' if sorix.cuda.is_available() else 'cpu'
device

✅ GPU basic operation passed
✅ GPU available: NVIDIA GeForce RTX 4070 Laptop GPU
CUDA runtime version: 13000
CuPy version: 13.6.0


'gpu'

## Linear + GPU

In [58]:
features = 5
classes = 2
examples = 10

X = tensor(np.random.randn(examples, features), device=device)
linear = Linear(features,classes).to(device)
linear(X)

Tensor(
[[-5.16703327 -3.59290969]
 [ 2.79656698  2.81765267]
 [ 1.39700557  1.11127165]
 [-1.22504713 -0.81609523]
 [ 0.15312325 -0.12838952]
 [-0.22987993 -0.93993913]
 [ 1.69610384 -1.71439391]
 [-0.35339305  0.65142666]
 [ 0.67518722 -0.39164053]
 [ 0.22420753 -0.41889924]], shape=(10, 2), device=gpu, requires_grad=True)

## Batch Norm + GPU

In [59]:
bn = BatchNorm1d(classes).to(device)
bn(linear(X))

Tensor(
[[-2.5330769  -2.00942872]
 [ 1.37349087  1.95325517]
 [ 0.68693186  0.89845714]
 [-0.59932389 -0.29294312]
 [ 0.07674169  0.1321616 ]
 [-0.11114166 -0.36949713]
 [ 0.8336554  -0.84822572]
 [-0.17173139  0.61420433]
 [ 0.33284172 -0.0305668 ]
 [ 0.11161229 -0.04741675]], shape=(10, 2), device=gpu, requires_grad=True)

## ReLU + GPU

In [60]:
relu = ReLU()
ReLU(X)

Tensor(
[[0.         0.         0.         1.42350407 0.        ]
 [1.44277295 1.88202868 0.         0.         0.6079959 ]
 [0.         1.41685226 0.         0.         0.50730439]
 [0.55991084 0.         0.         0.52641969 0.        ]
 [0.16115509 0.53765079 0.         0.         0.        ]
 [0.         0.0046861  0.23670877 0.         0.18401204]
 [1.49590944 0.         0.         0.         0.85012796]
 [0.         0.75653219 0.48502291 0.         0.        ]
 [1.22528896 0.         0.50027597 0.         0.87791816]
 [0.         0.13026655 0.         0.         0.        ]], shape=(10, 5), device=gpu, requires_grad=False)

## Linear + ReLU + Batch Norm + GPU

In [61]:
ReLU(bn(linear(X)))

Tensor(
[[0.         0.        ]
 [1.37349087 1.95325517]
 [0.68693186 0.89845714]
 [0.         0.        ]
 [0.07674169 0.1321616 ]
 [0.         0.        ]
 [0.8336554  0.        ]
 [0.         0.61420433]
 [0.33284172 0.        ]
 [0.11161229 0.        ]], shape=(10, 2), device=gpu, requires_grad=True)